In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
df = pd.read_csv("/content/dataset-first.csv")
df.head()

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return
0,0,RELIANCE,Reliance Financial Secures Strategic Partnersh...,2025-05-27,1422.099976,-0.646927,-0.084379,-0.590678
1,1,RELIANCE,"Reliance, financials lead rise in Indian share...",2025-06-26,1495.300049,1.344210,0.354439,2.213601
2,2,RELIANCE,Jio BlackRock Gets SEBI Approval as Stockbroke...,2025-06-27,1515.400024,-0.976643,0.857859,0.224365
3,3,RELIANCE,Reliance Media works Financial Services Pvt re...,2025-06-26,1495.300049,1.344210,0.354439,2.213601
4,4,RELIANCE,Reliance on informal finance persists despite ...,2025-06-22,1456.800049,-0.411862,0.720758,4.022513


Getting Rid OF Outliers

In [8]:
df = df[~(df["1d_return"].abs() > 42)]
df = df[~(df["3d_return"].abs() > 42)]
df = df[~(df["5d_return"].abs() > 42)]
df.head()

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return
0,0,RELIANCE,Reliance Financial Secures Strategic Partnersh...,2025-05-27,1422.099976,-0.646927,-0.084379,-0.590678
1,1,RELIANCE,"Reliance, financials lead rise in Indian share...",2025-06-26,1495.300049,1.344210,0.354439,2.213601
2,2,RELIANCE,Jio BlackRock Gets SEBI Approval as Stockbroke...,2025-06-27,1515.400024,-0.976643,0.857859,0.224365
3,3,RELIANCE,Reliance Media works Financial Services Pvt re...,2025-06-26,1495.300049,1.344210,0.354439,2.213601
4,4,RELIANCE,Reliance on informal finance persists despite ...,2025-06-22,1456.800049,-0.411862,0.720758,4.022513


creating a new column which would have highest return value amoung mentioned returns

In [9]:
df["max_abs_return"] = df[["1d_return", "3d_return", "5d_return"]].abs().max(axis=1)

In [10]:
df.sort_values(by="max_abs_return", ascending=False).head(5)

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return
6266,6266,ADANIENSOL,AGEL clarifies on TotalEnergies' update: Adani...,2024-11-26,600.750000,9.995837,39.908448,34.332085,39.908448
6275,6275,ADANIENSOL,Adani Energy Solutions shares hit 52-week low!...,2024-11-25,625.200012,-3.910750,16.258791,29.078692,29.078692
6260,6260,ADANIENSOL,Why Are Adani Shares Falling? - Forbes,2024-11-18,866.549988,0.577001,-19.537244,-27.851824,27.851824
3168,3168,INDUSINDBK,"Mutual funds take Rs 6,900-crore hit as IndusI...",2025-03-10,900.500000,-27.157134,-25.335927,-24.775122,27.157134
3172,3172,INDUSINDBK,IndusInd Bank's shares slide; CEO says quarter...,2025-03-10,900.500000,-27.157134,-25.335927,-24.775122,27.157134


In [11]:
print(len(df))

6904


In [12]:

from transformers import pipeline
sentiment = pipeline("sentiment-analysis", model="ahmedrachid/FinancialBERT-Sentiment-Analysis", device=0)
sentiment(["Markets rally after Fed decision", "Company reports unexpected losses"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'positive', 'score': 0.9992377758026123},
 {'label': 'neutral', 'score': 0.7964394092559814}]

In [13]:
df_headlines = df["Headline"]
df_headlines.head()

,Headline
0,Reliance Financial Secures Strategic Partnersh...
1,"Reliance, financials lead rise in Indian share..."
2,Jio BlackRock Gets SEBI Approval as Stockbroke...
3,Reliance Media works Financial Services Pvt re...
4,Reliance on informal finance persists despite ...


In [14]:
df_headlines = df_headlines.to_list()
print(df_headlines[:5])

['Reliance Financial Secures Strategic Partnership to Accelerate Growth in Revenue-Based Financing - Business Wire', 'Reliance, financials lead rise in Indian shares - TradingView', 'Jio BlackRock Gets SEBI Approval as Stockbroker; Reliance Backed Jio Financial Service Share Surges Nearly 4% - Goodreturns', 'Reliance Media works Financial Services Pvt reports standalone net loss of Rs 9.13 crore in the March 2025 quarter - Business Standard', 'Reliance on informal finance persists despite multiple financial inclusion measures: Report - The Economic Times']


In [15]:
df_headlines_labels = sentiment(df_headlines)

In [16]:
df_headlines_labels_two = df_headlines_labels.copy()

In [17]:
df_top_headlines = df[(df["1d_return"].abs() > 3)] # getting top headlines
df_top_headlines.head()

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return
14,14,RELIANCE,Reliance Industries Q4 Results Highlights: RIL...,2025-04-25,1300.400024,5.259922,7.636106,8.043677,8.043677
99,99,RELIANCE,Reliance Capital lenders accuse IIHL of non-co...,2024-08-02,1494.253418,-3.468227,-2.886285,-2.301035,3.468227
124,124,TCS,Strong TCV in a seasonally challenging Q3 posi...,2025-01-09,3993.750488,5.615451,6.245586,4.808294,6.245586
157,157,TCS,"TCS offers 3 free financial courses for all, h...",2025-01-09,3993.750488,5.615451,6.245586,4.808281,6.245586
171,171,TCS,"Stock market fall: TCS, Jio Financial, Asian P...",2025-02-27,3580.723389,-3.579190,-3.307914,-2.228342,3.579190


In [18]:
len(df_top_headlines)

797

In [19]:
df_headlines_labels_two[-5:]

[{'label': 'neutral', 'score': 0.9982548356056213},
 {'label': 'neutral', 'score': 0.9987384676933289},
 {'label': 'positive', 'score': 0.9754828214645386},
 {'label': 'neutral', 'score': 0.9985639452934265},
 {'label': 'positive', 'score': 0.9428244233131409}]

In [20]:
df_headlines_labels_two = pd.DataFrame(df_headlines_labels_two) # converting the list to DataFrame
df_headlines_labels_two["Headline"] = df["Headline"]
df_headlines_labels_two.head()

,label,score,Headline
0,neutral,0.997951,Reliance Financial Secures Strategic Partnersh...
1,positive,0.999809,"Reliance, financials lead rise in Indian share..."
2,neutral,0.999529,Jio BlackRock Gets SEBI Approval as Stockbroke...
3,negative,0.988214,Reliance Media works Financial Services Pvt re...
4,neutral,0.997900,Reliance on informal finance persists despite ...


In [21]:
df_headlines_labels_two.to_csv("Sentiments.csv",index = False)

In [22]:
print(len(df),len(df_headlines_labels_two))

6904 6904


In [23]:
merged_df = pd.merge(df, df_headlines_labels_two, on='Headline', how='inner') # merging scores and df
merged_df.head()

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return,label,score
0,0,RELIANCE,Reliance Financial Secures Strategic Partnersh...,2025-05-27,1422.099976,-0.646927,-0.084379,-0.590678,0.646927,neutral,0.997951
1,1,RELIANCE,"Reliance, financials lead rise in Indian share...",2025-06-26,1495.300049,1.344210,0.354439,2.213601,2.213601,positive,0.999809
2,2,RELIANCE,Jio BlackRock Gets SEBI Approval as Stockbroke...,2025-06-27,1515.400024,-0.976643,0.857859,0.224365,0.976643,neutral,0.999529
3,3,RELIANCE,Reliance Media works Financial Services Pvt re...,2025-06-26,1495.300049,1.344210,0.354439,2.213601,2.213601,negative,0.988214
4,4,RELIANCE,Reliance on informal finance persists despite ...,2025-06-22,1456.800049,-0.411862,0.720758,4.022513,4.022513,neutral,0.997900


In [24]:
neutral_mismatches = merged_df[
    (merged_df["label"].str.lower() == "neutral") &
    (merged_df["max_abs_return"].abs() > 3)
]
neutral_mismatches.head() # here stocks showed high returns(-ve or +ve) in a short span but headlines were mentioned neutral,its because the model for sentiment analysis is trained on UK,US stock info not on india,we will be fixing that

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return,label,score
4,4,RELIANCE,Reliance on informal finance persists despite ...,2025-06-22,1456.800049,-0.411862,0.720758,4.022513,4.022513,neutral,0.997900
7,7,RELIANCE,Reliance Capital finally comes under Hinduja f...,2025-03-19,1247.150024,1.764022,4.406042,3.070996,4.406042,neutral,0.999835
9,9,RELIANCE,Will keep Reliance Capital’s key entities as m...,2025-03-19,1247.150024,1.764022,4.406042,3.070996,4.406042,neutral,0.999803
12,12,RELIANCE,IndusInd International to list insurance units...,2025-03-18,1238.849976,0.669980,3.027001,5.105542,5.105542,neutral,0.999838
16,16,RELIANCE,RCap lenders withdraw petition against IndusIn...,2025-03-30,1252.599976,-0.115755,-0.311354,-3.824048,3.824048,neutral,0.998991


In [25]:
len(neutral_mismatches)

2573

In [26]:
neutral_mismatches.sort_values(by="max_abs_return", ascending=False).head(50)

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return,label,score
7480,6266,ADANIENSOL,AGEL clarifies on TotalEnergies' update: Adani...,2024-11-26,600.750000,9.995837,39.908448,34.332085,39.908448,neutral,0.611113
7489,6275,ADANIENSOL,Adani Energy Solutions shares hit 52-week low!...,2024-11-25,625.200012,-3.910750,16.258791,29.078692,29.078692,neutral,0.998946
2562,2166,ADANIPORTS,Fundamental Analysis of Adani Ports – Industry...,2023-01-22,756.968079,-1.020733,-7.268707,-22.378262,22.378262,neutral,0.998042
5840,4815,CONCOR,Rail PSUs under scrutiny for giving unauthoriz...,2024-06-02,1154.349365,-19.203743,-15.414094,-10.172433,19.203743,neutral,0.996104
3808,3151,INDUSINDBK,"Q2 results today: ITC, IndusInd Bank, NTPC, Bi...",2024-10-24,1280.050049,-18.628184,-17.596968,-18.897701,18.897701,neutral,0.999574
7787,6540,ESCORTS,"This CFP says helping strippers, prostitutes a...",2022-04-06,1859.946777,-9.217215,-15.113269,-18.511167,18.511167,neutral,0.999520
7659,6425,COFORGE,COFORGE LIMITED Shares Tumble 4.11% Amid Volum...,2025-04-01,1556.160034,-0.247402,-15.074290,-18.508380,18.508380,neutral,0.999680
7679,6444,COFORGE,Mid-tier IT firms reap gains from smaller deal...,2025-03-31,1556.160034,-0.247402,-8.012033,-18.508380,18.508380,neutral,0.998576
6178,5109,INDUSTOWER,"Stocks To Watch: TCS, Adani Group, Avenue Supe...",2025-01-13,320.500000,6.177843,10.624021,17.191890,17.191890,neutral,0.999586
7352,6150,MOTHERSON,Analyst corner: Motherson Sumi expects current...,2020-04-17,34.568211,-2.419341,-17.108296,-15.725798,17.108296,neutral,0.999754


In [27]:
neutral_mismatches.to_csv("neutral_mismatches", index = False)

Manual Review of 1000 mismatched sentimens for indian Stcok Market

model will correct the rest after fine tunning

In [28]:
neutral_mismatches = pd.read_csv("/content/neutral_mismatches")
neutral_mismatches=neutral_mismatches.sort_values(by="max_abs_return", ascending=False)
manual_labels = neutral_mismatches.head(1200).copy() # 1200 coz during labeliing I saw many common headlines and we will be droping rows with duplicate headlines

In [29]:
headlines = manual_labels["Headline"].to_list()
len(headlines)

1200

In [30]:
headlines[:5]

["AGEL clarifies on TotalEnergies' update: Adani Green, Adani Energy & Adani Total shares slump up to 8% - Business Today",
 'Adani Energy Solutions shares hit 52-week low! Stock down 28% in 3 sessions; more pain ahead? - Business Today',
 'Fundamental Analysis of Adani Ports – Industry, Financials & More - Trade Brains',
 'Rail PSUs under scrutiny for giving unauthorized financial incentives - The Sunday Guardian Live',
 'Q2 results today: ITC, IndusInd Bank, NTPC, Bikaji Foods among others to declare earnings on October 24; check list - Upstox']

In [31]:
correct_sentiments = []
i=0
for i in range(len(headlines)):
  print(headlines[i])
  val = input()
  if val == "q":
    break
  elif val == "n":
    val = "negative"
  elif val == "p":
    val = "positive"
  else:
    val = "neutral"
  correct_sentiments.append(val) # ill not be doing it in 1 sitting,itll take time

AGEL clarifies on TotalEnergies' update: Adani Green, Adani Energy & Adani Total shares slump up to 8% - Business Today
q


In [32]:
manual_labels["Healine"] = headlines
manual_labels["Correct_lables"] = correct_sentiments

ValueError: Length of values (0) does not match length of index (1200)

In [33]:
manual_labels.to_csv("manual_labels.csv")

In [39]:
manual_labels = pd.read_csv("/content/manual_labels.csv")
len(manual_labels)

1200

In [40]:
manual_labels.head()

,Unnamed: 0,Headline,Correct_labels
0,0,AGEL clarifies on TotalEnergies' update: Adani...,negative
1,1,Adani Energy Solutions shares hit 52-week low!...,negative
2,2,Fundamental Analysis of Adani Ports – Industry...,neutral
3,3,Rail PSUs under scrutiny for giving unauthoriz...,negative
4,4,"Q2 results today: ITC, IndusInd Bank, NTPC, Bi...",neutral


In [41]:
manual_labels["Correct_labels"].value_counts()

,count
Correct_labels,
neutral,686
positive,409
negative,105


In [42]:
df_part = neutral_mismatches.head(1200).copy().reset_index(drop=True)
labels = manual_labels["Correct_labels"].reset_index(drop=True)

correct_label_df = pd.concat([df_part, labels], axis=1)

In [43]:
correct_label_df.head()

,Unnamed: 0,Ticker,Headline,Date,Close,1d_return,3d_return,5d_return,max_abs_return,label,score,Correct_labels
0,6266,ADANIENSOL,AGEL clarifies on TotalEnergies' update: Adani...,2024-11-26,600.750000,9.995837,39.908448,34.332085,39.908448,neutral,0.611113,negative
1,6275,ADANIENSOL,Adani Energy Solutions shares hit 52-week low!...,2024-11-25,625.200012,-3.910750,16.258791,29.078692,29.078692,neutral,0.998946,negative
2,2166,ADANIPORTS,Fundamental Analysis of Adani Ports – Industry...,2023-01-22,756.968079,-1.020733,-7.268707,-22.378262,22.378262,neutral,0.998042,neutral
3,4815,CONCOR,Rail PSUs under scrutiny for giving unauthoriz...,2024-06-02,1154.349365,-19.203743,-15.414094,-10.172433,19.203743,neutral,0.996104,negative
4,3151,INDUSINDBK,"Q2 results today: ITC, IndusInd Bank, NTPC, Bi...",2024-10-24,1280.050049,-18.628184,-17.596968,-18.897701,18.897701,neutral,0.999574,neutral


In [44]:
correct_label_df["Correct_labels"].value_counts()

,count
Correct_labels,
neutral,686
positive,409
negative,105


In [45]:
correct_label_df=correct_label_df.drop_duplicates(subset="Headline") #many headlines were common so we are dropping them,will drop more when entire dataset is complete

In [46]:
len(correct_label_df)

973

In [47]:
correct_label_df.to_csv("correct_labels.csv",index=False)

In [48]:
correct_label_df_example = correct_label_df.head(50).copy()

In [49]:
correct_label_df_example.to_csv("correct_labels_example.csv",index=False)

In [51]:
from google.colab import files
files.download('03_data_cleaning_sentiment_scoring.ipynb')

FileNotFoundError: Cannot find file: 03_data_cleaning_sentiment_scoring.ipynb

In [52]:
!jupyter nbconvert --to notebook --ClearMetadataPreprocessor.enabled=True \
  --output=clean_notebook.ipynb 03_data_cleaning_sentiment_scoring.ipynb

[NbConvertApp] WARNING | pattern '03_data_cleaning_sentiment_scoring.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_

In [54]:
from google.colab import files
files.download('03_data_cleaning_sentiment_scoring.ipynb')

FileNotFoundError: Cannot find file: 03_data_cleaning_sentiment_scoring.ipynb